In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats
import gseapy as gp
import re
import sys 

import cptac
import cptac.utils as u

/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Load cancer data

In [2]:
en = cptac.Endometrial()
# br = cptac.Brca()
# cl = cptac.Colon()

In [3]:
gene = 'PIK3CA'

# Endometrial

## Filter out hotspot mutations
Hotspots are:
E542K
E545K
H1047R


#### Get the mutation type, and proteomics for PIK3CA

In [4]:
# mut = en.get_genotype_all_vars(gene, mutation_hotspot=['E542K', 'E535K', 'H1047R'])
# prot = en.get_proteomics()
# protfilter = prot.index.str.endswith('.N')
# prot = prot[~protfilter]



In [5]:
# joined = mut.join(prot)
# joined

In [6]:
# joined[joined.Location.str.contains('H1047R')]

In [4]:
mut = en.get_genotype_all_vars(gene, mutation_hotspot=['E542K', 'E545K', 'H1047R'])
prot = en.get_proteomics(tissue_type="tumor")

cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 48 samples for the PIK3CA gene (/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/cptac/dataset.py, line 311)


In [8]:
mut["Mutation"].unique()

array(['Missense_Mutation', 'Wildtype_Tumor', 'Amplification',
       'Frame_Shift_Ins', 'In_Frame_Del', 'Deletion'], dtype=object)

#### Join mutation type and proteomics together

In [5]:
joined = mut.join(prot)

In [6]:
joined['Mutation'].unique()

array(['Missense_Mutation', 'Wildtype_Tumor', 'Amplification',
       'Frame_Shift_Ins', 'In_Frame_Del', 'Deletion'], dtype=object)

In [11]:
mut = en.get_genotype_all_vars(gene, mutation_hotspot=['E542K', 'E535K', 'H1047R'])
prot = en.get_proteomics()
protfilter = prot.index.str.endswith('.N')
prot = prot[~protfilter]

cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 48 samples for the PIK3CA gene (/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/cptac/dataset.py, line 394)


#### Select samples containing hotspot mutations

In [7]:
en_hotspot_df = joined[joined.Location.str.contains('E542K') | 
                    joined.Location.str.contains('E545K') |
                    joined.Location.str.contains('H1047R')]

In [8]:
en_hotspot_df

Name,Mutation,Location,Mutation_Status,A1BG,A2M,A2ML1,A4GALT,AAAS,AACS,AADAT,...,ZSWIM8,ZSWIM9,ZW10,ZWILCH,ZWINT,ZXDC,ZYG11B,ZYX,ZZEF1,ZZZ3
Patient_ID,,,,,,,,,,,,,,,,,,,,,
C3L-00006,Missense_Mutation,p.E545K,Single_mutation,-1.1800,-0.863,-0.802,0.22200,0.2560,0.6650,1.2800,...,-0.08770,NaN,0.0229,0.109,NaN,-0.3320,-0.4330,-1.0200,-0.1230,-0.08590
C3L-00032,Missense_Mutation,p.E545K,Single_mutation,-0.5280,-1.320,0.435,NaN,-0.2400,1.0400,-0.0213,...,0.00112,-0.1450,0.0105,-0.116,NaN,0.1510,-0.0740,-0.5400,0.3200,-0.41900
C3L-00362,Missense_Mutation,p.H1047R,Single_mutation,-0.9240,-0.445,1.570,-0.67800,0.1730,0.4360,0.2270,...,-0.04960,NaN,0.1080,-0.153,NaN,0.1090,-0.3360,-0.8220,-0.0338,0.12100
C3L-00601,Missense_Mutation,p.E545K,Multiple_mutation,-0.4540,-0.242,NaN,NaN,0.2580,0.2190,-0.2490,...,-0.15800,NaN,0.6670,1.300,0.441,0.1300,-0.0659,-0.9230,-0.1630,0.11200
C3L-00605,Missense_Mutation,p.E545K,Single_mutation,-0.2400,0.594,3.400,0.15400,0.0932,0.2830,-0.0789,...,-0.05110,-0.0136,0.2140,0.683,0.433,-0.2080,-0.3470,-0.9110,0.0692,-0.23200
C3L-00921,Missense_Mutation,p.E542K,Single_mutation,-1.1900,-1.190,3.660,1.27000,-0.1050,-0.0021,0.0813,...,0.09700,0.2540,0.1510,0.285,-0.239,-0.0681,-0.2180,-0.0873,-0.1270,0.37400
C3L-00947,Missense_Mutation,p.E545K,Single_mutation,-0.4850,0.917,0.304,-0.00675,0.1780,-0.4230,0.2240,...,0.04290,-0.2120,0.1690,0.521,-0.147,0.3540,0.1700,-1.0100,-0.1100,0.13100
C3N-00323,Missense_Mutation,p.E545K,Single_mutation,-0.8170,-1.130,NaN,NaN,0.1020,0.3710,0.6440,...,-0.02220,NaN,-0.2790,0.423,NaN,0.2940,-0.1300,-0.1060,-0.2990,0.38500
C3N-00324,Missense_Mutation,p.H1047R,Single_mutation,-0.4490,-0.440,1.150,NaN,-0.6450,0.3160,0.0809,...,0.04490,0.0452,-0.1520,-0.197,0.295,-0.1880,-0.2650,0.2500,-0.1130,-0.30200


#### Join the proteomics for the wildtype mutations to the hotspot mutations

In [9]:
wildtype  = joined.loc[joined.Mutation == "Wildtype_Tumor"]

endo_hotspot = en_hotspot_df.append(wildtype)

#### Drop unncecesary columns
The resulting dataframe is what we will be working with from this point forward. It contains the mutations type and the proteomics for each gene.

In [10]:
prot_and_mutations = endo_hotspot.drop(columns = ["Mutation_Status", "Location"])
prot_and_mutations

Name,Mutation,A1BG,A2M,A2ML1,A4GALT,AAAS,AACS,AADAT,AAED1,AAGAB,...,ZSWIM8,ZSWIM9,ZW10,ZWILCH,ZWINT,ZXDC,ZYG11B,ZYX,ZZEF1,ZZZ3
Patient_ID,,,,,,,,,,,,,,,,,,,,,
C3L-00006,Missense_Mutation,-1.1800,-0.86300,-0.8020,0.22200,0.2560,0.66500,1.28000,-0.33900,0.41200,...,-0.087700,NaN,0.02290,0.1090,NaN,-0.3320,-0.43300,-1.02000,-0.12300,-0.08590
C3L-00032,Missense_Mutation,-0.5280,-1.32000,0.4350,NaN,-0.2400,1.04000,-0.02130,-0.04790,0.41900,...,0.001120,-0.14500,0.01050,-0.1160,NaN,0.1510,-0.07400,-0.54000,0.32000,-0.41900
C3L-00362,Missense_Mutation,-0.9240,-0.44500,1.5700,-0.67800,0.1730,0.43600,0.22700,-0.00248,0.47900,...,-0.049600,NaN,0.10800,-0.1530,NaN,0.1090,-0.33600,-0.82200,-0.03380,0.12100
C3L-00601,Missense_Mutation,-0.4540,-0.24200,NaN,NaN,0.2580,0.21900,-0.24900,-1.33000,0.20400,...,-0.158000,NaN,0.66700,1.3000,0.4410,0.1300,-0.06590,-0.92300,-0.16300,0.11200
C3L-00605,Missense_Mutation,-0.2400,0.59400,3.4000,0.15400,0.0932,0.28300,-0.07890,-0.61100,0.37000,...,-0.051100,-0.01360,0.21400,0.6830,0.4330,-0.2080,-0.34700,-0.91100,0.06920,-0.23200
C3L-00921,Missense_Mutation,-1.1900,-1.19000,3.6600,1.27000,-0.1050,-0.00210,0.08130,-0.69100,-0.29500,...,0.097000,0.25400,0.15100,0.2850,-0.2390,-0.0681,-0.21800,-0.08730,-0.12700,0.37400
C3L-00947,Missense_Mutation,-0.4850,0.91700,0.3040,-0.00675,0.1780,-0.42300,0.22400,-0.24700,0.07870,...,0.042900,-0.21200,0.16900,0.5210,-0.1470,0.3540,0.17000,-1.01000,-0.11000,0.13100
C3N-00323,Missense_Mutation,-0.8170,-1.13000,NaN,NaN,0.1020,0.37100,0.64400,0.28100,0.11400,...,-0.022200,NaN,-0.27900,0.4230,NaN,0.2940,-0.13000,-0.10600,-0.29900,0.38500
C3N-00324,Missense_Mutation,-0.4490,-0.44000,1.1500,NaN,-0.6450,0.31600,0.08090,-0.47000,0.80400,...,0.044900,0.04520,-0.15200,-0.1970,0.2950,-0.1880,-0.26500,0.25000,-0.11300,-0.30200


## Run T-test and difference of Median for each cancer type

#### Get the median proteomic expression among all samples, for each gene.
This is done for missense mutations and wildtype separately


In [16]:
missense = prot_and_mutations[prot_and_mutations.Mutation == "Missense_Mutation"] #get all missense_mutations
wt = prot_and_mutations[prot_and_mutations.Mutation == "Wildtype_Tumor"] #get all wildtype mutations
missense_med = missense.median() #get the medain of the missense. This will give the median expression among all samples, for each gene
wt_med = wt.median() #get the median of the wildtype

#### Get the medain difference between missense and wildtype for each gene.
This is done by (for each gene) subtracting the median of the missense mutations from the median of the wildtype.
This means that if the difference is positive the proteomics of the wildtype have higher expression than the missense mutation. If the difference is negative then the missense mutation has a higher expression than the wildtype.

In [17]:
endo_d = {}

for gene in prot_and_mutations:
    if gene == "Mutation": continue
    dif_endo =  missense_med[gene] - wt_med[gene]
    endo_d[gene] = dif_endo

median_diff = pd.DataFrame.from_dict(endo_d, orient='index', columns=['Difference_In_Median'])
median_diff.head()

,Difference_In_Median
A1BG,0.055500
A2M,0.038000
A2ML1,1.105850
A4GALT,0.111075
AAAS,0.156100


In [12]:
prot_and_mutations

Name,Mutation,A1BG,A2M,A2ML1,A4GALT,AAAS,AACS,AADAT,AAED1,AAGAB,...,ZSWIM8,ZSWIM9,ZW10,ZWILCH,ZWINT,ZXDC,ZYG11B,ZYX,ZZEF1,ZZZ3
Patient_ID,,,,,,,,,,,,,,,,,,,,,
C3L-00006,Missense_Mutation,-1.1800,-0.86300,-0.8020,0.22200,0.2560,0.66500,1.28000,-0.33900,0.41200,...,-0.087700,NaN,0.02290,0.1090,NaN,-0.3320,-0.43300,-1.02000,-0.12300,-0.08590
C3L-00032,Missense_Mutation,-0.5280,-1.32000,0.4350,NaN,-0.2400,1.04000,-0.02130,-0.04790,0.41900,...,0.001120,-0.14500,0.01050,-0.1160,NaN,0.1510,-0.07400,-0.54000,0.32000,-0.41900
C3L-00362,Missense_Mutation,-0.9240,-0.44500,1.5700,-0.67800,0.1730,0.43600,0.22700,-0.00248,0.47900,...,-0.049600,NaN,0.10800,-0.1530,NaN,0.1090,-0.33600,-0.82200,-0.03380,0.12100
C3L-00601,Missense_Mutation,-0.4540,-0.24200,NaN,NaN,0.2580,0.21900,-0.24900,-1.33000,0.20400,...,-0.158000,NaN,0.66700,1.3000,0.4410,0.1300,-0.06590,-0.92300,-0.16300,0.11200
C3L-00605,Missense_Mutation,-0.2400,0.59400,3.4000,0.15400,0.0932,0.28300,-0.07890,-0.61100,0.37000,...,-0.051100,-0.01360,0.21400,0.6830,0.4330,-0.2080,-0.34700,-0.91100,0.06920,-0.23200
C3L-00921,Missense_Mutation,-1.1900,-1.19000,3.6600,1.27000,-0.1050,-0.00210,0.08130,-0.69100,-0.29500,...,0.097000,0.25400,0.15100,0.2850,-0.2390,-0.0681,-0.21800,-0.08730,-0.12700,0.37400
C3L-00947,Missense_Mutation,-0.4850,0.91700,0.3040,-0.00675,0.1780,-0.42300,0.22400,-0.24700,0.07870,...,0.042900,-0.21200,0.16900,0.5210,-0.1470,0.3540,0.17000,-1.01000,-0.11000,0.13100
C3N-00323,Missense_Mutation,-0.8170,-1.13000,NaN,NaN,0.1020,0.37100,0.64400,0.28100,0.11400,...,-0.022200,NaN,-0.27900,0.4230,NaN,0.2940,-0.13000,-0.10600,-0.29900,0.38500
C3N-00324,Missense_Mutation,-0.4490,-0.44000,1.1500,NaN,-0.6450,0.31600,0.08090,-0.47000,0.80400,...,0.044900,0.04520,-0.15200,-0.1970,0.2950,-0.1880,-0.26500,0.25000,-0.11300,-0.30200


#### Do a t-test for every gene.

In [11]:
genes = list(prot_and_mutations.columns[1:])
n = u.wrap_ttest(prot_and_mutations, 'Mutation', correction_method="fdr_bh")
print(n)

/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3367: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/statsmodels/stats/multitest.py:326: RuntimeWarning: invalid value encountered in less_equal
  reject = pvals_sorted <= ecdffactor*alpha


None


In [ ]:
prot_and_mutations

In [23]:
import pandas as pd
import numpy as np
import scipy.stats
import statsmodels.stats.multitest
import operator


Help on function wrap_ttest in module cptac.utils:

wrap_ttest(df, label_column, comparison_columns=None, alpha=0.05, return_all=False, correction_method='bonferroni')



In [14]:
def wrap_ttest(df, label_column, comparison_columns=None, alpha=.05, return_all=False, correction_method='bonferroni'):
    try:
        '''Verify precondition that label column exists and has exactly 2 unique values'''
        label_values = df[label_column].unique()
        if len(label_values) != 2:
            print("Incorrectly Formatted Dataframe! Label column must have exactly 2 unique values.")
            return None
        
        '''Partition dataframe into two sets, one for each of the two unique values from the label column'''
        partition1 = df.loc[df[label_column] == label_values[0]]
        partition2 = df.loc[df[label_column] == label_values[1]]

        '''If no comparison columns specified, use all columns except the specified labed column'''
        if not comparison_columns:
            comparison_columns = list(df.columns)
            comparison_columns.remove(label_column)

        '''Determine the number of real valued columns on which we will do t-tests'''
        number_of_comparisons = len(comparison_columns)

        '''Store comparisons and p-values in two arrays'''
        comparisons = []
        pvals = []
        
        '''Loop through each comparison column, perform the t-test, and record the p-val'''
        for column in comparison_columns:
            if len(partition1[column].dropna(axis=0)) <= 1:
                continue
            elif len(partition2[column].dropna(axis=0)) <= 1:
                continue
            else:
                stat, pval = scipy.stats.ttest_ind(partition1[column].dropna(axis=0), partition2[column].dropna(axis=0))
                comparisons.append(column)
                pvals.append(pval)
            
        import pdb; pdb.set_trace()
        '''Correct for multiple testing to determine if each comparison meets the new cutoff'''
        results = statsmodels.stats.multitest.multipletests(pvals=pvals, alpha=alpha, method=correction_method)
        reject = results[0]

        '''Format results in a pandas dataframe'''
        results_df = pd.DataFrame(columns=['Comparison','P_Value'])

        '''If return all, add all comparisons and p-values to dataframe'''
        if return_all:
            results_df['Comparison'] = comparisons
            results_df['P_Value'] = pvals

            '''Else only add significant comparisons'''
        else:
            for i in range(0, len(reject)):
                if reject[i]:
                    results_df = results_df.append({'Comparison':comparisons[i],'P_Value':pvals[i]}, ignore_index=True)


        '''Sort dataframe by ascending p-value'''
        results_df = results_df.sort_values(by='P_Value', ascending=True)
        results_df = results_df.reset_index(drop=True)

        '''If results df is not empty, return it, else return None'''
        if len(results_df) > 0:
            return results_df
        else:
            return None


    except:
        print("Incorrectly Formatted Dataframe!")
        return None

In [33]:
wrap_ttest(prot_and_mutations, label_column='Mutation', correction_method="fdr_bh")

> <ipython-input-32-e05b3ce53aee>(33)wrap_ttest()
-> results = statsmodels.stats.multitest.multipletests(pvals=pvals, alpha=alpha, method=correction_method)
(Pdb) pvals
[0.4346455726866979, 0.7142152592933092, 0.03704373350775898, 0.8370213135074317, 0.6645449527922938, 0.7327137413841396, 0.6906131204398366, 0.12319346167202375, 0.13963170285390072, 0.06460423034722369, 0.7429494499920608, 0.39619390771264695, 0.7632335365850501, 0.41424033193069254, 0.34939866038458534, 0.647126110945794, 0.4288843822250653, 0.2571596661204336, 0.9154816735615795, 0.328961368781743, 0.33473387925795794, 0.1444418069661645, 0.49980271506414764, 0.8193643280175754, 0.05752321810282647, 0.13555000140197718, 0.4019837053426042, 0.7652328601775571, 0.827863177288731, 0.19886115106290247, 0.4664832310510957, 0.9041272988329425, 0.45725813173530516, 0.13819908169293293, 0.7007309144639857, 0.6622659933042976, 0.8772353747571924, 0.1760513900132739, 0.5528773471566142, 0.556112768039748, 0.972151196043904, 0

(Pdb) results
*** NameError: name 'results' is not defined
(Pdb) results
*** NameError: name 'results' is not defined
(Pdb) n
> <ipython-input-32-e05b3ce53aee>(34)wrap_ttest()
-> reject = results[0]
(Pdb) results
(array([False, False, False, ..., False, False, False]), array([nan, nan, nan, ..., nan, nan, nan]), 4.663439838736849e-06, 4.545867806164197e-06)
(Pdb) n
> <ipython-input-32-e05b3ce53aee>(37)wrap_ttest()
-> results_df = pd.DataFrame(columns=['Comparison','P_Value'])
(Pdb) reject
array([False, False, False, ..., False, False, False])
(Pdb) np.where(reject)
(array([], dtype=int64),)
(Pdb) q
Incorrectly Formatted Dataframe!


In [ ]:
t_test = t_test.set_index('Comparison')
t_test.head()

#### Join difference in median and t-test p value into the same dataframe

In [ ]:
# Step 4 join median and p value together
endo_df = median_diff.join(t_test)
endo_df.head()

# Brca
The Brca data uses different database_IDs, giving us multiple proteomic values for each gene. To deal with this, we will first separate genes that have multiple proteomics values recoreded (multiple database_IDs) from ones that don't.

## Dealing with multiple database_IDs

#### Separate genes with multiple database_IDs from genes with multiple database_IDs

In [ ]:
proteomics = br.get_proteomics(tissue_type="tumor")
fil = proteomics.columns.get_level_values('Name').duplicated(False)
duplicates = proteomics[proteomics.columns[fil]]
no_duplicates = proteomics[proteomics.columns[~fil]]
no_duplicates

### Dealing with genes that don't have multiple database_IDs
These will be processed the same way the data for endometrial was.

In [ ]:
#FOR NO DUPLICATES
#Flatten the multiindex (since there are no duplicate gene names, this won't be a problem)
no_duplicates = br.reduce_multiindex(no_duplicates, flatten=True)

#get mutation_tyoe
mut_type = br.get_genotype_all_vars('PIK3CA')

#join proteomics and mutation type
joined = mut_type.join(no_duplicates)

#select samples containing hotspot mutations
br_hotspot_df = joined[joined.Location.str.contains('E542K') | 
                    joined.Location.str.contains('E545K') |
                    joined.Location.str.contains('H1047R')]


#join wildtype proteomics to missense mutation proteomics 
wildtype  = joined.loc[joined.Mutation == "Wildtype_Tumor"]
brca_hotspot = br_hotspot_df.append(wildtype)

#drop unncessary columns
prot_and_mutations = brca_hotspot.drop(columns = ["Mutation_Status", "Location"])

# Get the difference in medians
missense = prot_and_mutations[prot_and_mutations.Mutation == "Missense_Mutation"] #get all missense_mutations
wt = prot_and_mutations[prot_and_mutations.Mutation == "Wildtype_Tumor"] #getn all wildtype
missense_med = missense.median() #get the medain of the missense. This will give the median expression among all samples, for each gene
wt_med = wt.median() #get the median of the wildtype


no_dup_d = {}

for prot in no_duplicates:
    dif_brca = missense_med[prot] - wt_med[prot]
    no_dup_d[prot] = dif_brca

median_diff = pd.DataFrame.from_dict(no_dup_d, orient='index', columns=['Difference_In_Median'])

#Do a t test for every gene and report the p-value
genes = list(prot_and_mutations.columns[1:])
t_test = u.wrap_ttest(prot_and_mutations, 'Mutation', genes, correction_method= "fdr_bh")
t_test = t_test.set_index('Comparison')

#join median and p value together
no_dup_df = median_diff.join(t_test)
no_dup_df.head()

## Dealing with genes that do have multiple database_IDs

#### Get t-test results and median difference

In [ ]:
#DEALING WITH DUPLICATES
#flatten the multiindex
duplicates = br.reduce_multiindex(duplicates, flatten=True)

# We need to figure which of databaseID we want to use, for each protein
proteomics = br.get_proteomics(tissue_type="tumor")
fil = proteomics.columns.get_level_values('Name').duplicated(False)
duplicates = proteomics[proteomics.columns[fil]]
duplicate_gene_names = list(set(duplicates.columns.get_level_values('Name'))) #get a list of genes names that are duplicated
duplicates = br.reduce_multiindex(duplicates, flatten=True)
duplicates

#  Step 1 - Create dataframe in order to do comparisons with wrap_ttest

mut_type = br.get_genotype_all_vars("PIK3CA")


joined = mut_type.join(duplicates)

#select samples containing hotspot mutations
br_hotspot_df = joined[joined.Location.str.contains('E542K') | 
                    joined.Location.str.contains('E545K') |
                    joined.Location.str.contains('H1047R')]


#join wildtype proteomics to missense mutation proteomics 
wildtype  = joined.loc[joined.Mutation == "Wildtype_Tumor"]
brca_hotspot = br_hotspot_df.append(wildtype)

#drop unncessary columns
prot_and_mutations = brca_hotspot.drop(columns = ["Mutation_Status", "Location"])


# get the difference in medians
missense = prot_and_mutations[prot_and_mutations.Mutation == "Missense_Mutation"] #get all missense_mutations
wt = prot_and_mutations[prot_and_mutations.Mutation == "Wildtype_Tumor"] #getn all wildtype
missense_med = missense.median() #get the medain of the missense. This will give the median expression among all samples, for each gene
wt_med = wt.median() #get the median of the wildtype


duplicates_d = {}

for prot in duplicates:
    dif_brca =  missense_med[prot] - wt_med[prot]
    duplicates_d[prot] = dif_brca
    

median_diff = pd.DataFrame.from_dict(duplicates_d, orient='index', columns=['Difference_In_Median'])


#step 3 do a t test for every gene and report the p-value
cols = list(prot_and_mutations.columns[1:])
t_test = u.wrap_ttest(prot_and_mutations, 'Mutation', cols, correction_method= "fdr_bh")
t_test = t_test.set_index('Comparison')

# Step 4 join median and p value together
dup_df = median_diff.join(t_test)
dup_df.head()

#### Select the database_ID that gives a lower p value from the t-test, and append selected database_ID to the no_duplicate dataframe

In [ ]:
#for each duplicated gene name select the one with the min p val and append to no_duplicat df
for name in duplicate_gene_names:
    gene_duplicate = dup_df[dup_df.index.str.contains(name)]
    #select the one with the min p value for t test
    min_row = gene_duplicate[gene_duplicate.P_Value == gene_duplicate.P_Value.min()]
    #append the selected version onto no_duplicate dataframe
    no_dup_df=no_dup_df.append(min_row)
no_dup_df=no_dup_df.sort_index()
no_dup_df.head()

#### Reformat the table by getting rid of databaseIDs so we can join it to Endo and Brca

In [ ]:
split = no_dup_df.index.str.split('_', 1).str[0]
no_dup_df['new_index']=split
brca_df = no_dup_df.set_index("new_index")
brca_df.head()

# Colon

In [ ]:
#  Step 1 - Create dataframe in order to do comparisons with wrap_ttest
proteomics = cl.get_proteomics(tissue_type="tumor")

prot_list = list(proteomics.columns)

mut_type = cl.get_genotype_all_vars("PIK3CA")


joined = mut_type.join(proteomics)

#select samples containing hotspot mutations
cl_hotspot_df = joined[joined.Location.str.contains('E542K') | 
                    joined.Location.str.contains('E545K') |
                    joined.Location.str.contains('H1047R')]

#join wildtype proteomics to missense mutation proteomics 
wildtype  = joined.loc[joined.Mutation == "Wildtype_Tumor"]
colon_hotspot = cl_hotspot_df.append(wildtype)

#drop unncessary columns
prot_and_mutations = colon_hotspot.drop(columns = ["Mutation_Status", "Location"])


#get the difference in medians
missense = prot_and_mutations[prot_and_mutations.Mutation == "nonsynonymous SNV"] #get all missense_mutations
wt = prot_and_mutations[prot_and_mutations.Mutation == "Wildtype_Tumor"] #getn all wildtype
missense_med = missense.median() #get the medain of the missense. This will give the median expression among all samples, for each gene
wt_med = wt.median() #get the median of the wildtype


colon_d = {}

for prot in proteomics:
    dif_colon = missense_med[prot] - wt_med[prot]
    colon_d[prot] = dif_colon

median_diff = pd.DataFrame.from_dict(colon_d, orient='index', columns=['Difference_In_Median'])

#do a t test for every gene and report the p-value
cols = list(prot_and_mutations.columns[1:])
t_test = u.wrap_ttest(prot_and_mutations, 'Mutation', cols, correction_method= "fdr_bh")
t_test = t_test.set_index('Comparison')

In [ ]:
t_test.head(30)

In [ ]:
#join median and p value together
colon_df = median_diff.join(t_test)
colon_df.head()

# Combine Endo, Brca, and Colon into one table

#### Combine brca and endo

In [ ]:
brca_df=brca_df.rename_axis(None)
brca_endo = brca_df.join(endo_df, lsuffix='_Brca', rsuffix='_Endo')
brca_endo.head()

#### Combine colon

In [ ]:
final_table = brca_endo.join(colon_df, rsuffix='_colon')
final_table.head()

#### Only rows that contain a significant pvalue

In [ ]:
significant = final_table.loc[(final_table['P_Value_Brca'] <= .05) |
                (final_table['P_Value_Endo'] <= .05) |
                (final_table['P_Value'] <= .05)]
significant.head()

In [ ]:
significant.to_csv("~/WhenMutationsDontMatter/PIK3CA/csv_files/proteomics.csv")